In [111]:
from lets_plot import *
import math
import numpy as np

In [112]:
LetsPlot.setup_html()

In [113]:
np.random.seed(42)

x = list(range(210, 1000, 97))

# Base rate increases with x (e.g., linear growth for lambda)
lambdas = [5 + (xi - 210) / 100 for xi in x]

# y values from Poisson distribution with increasing lambda
y = [np.random.poisson(lam) for lam in lambdas]

# Optional: scale and shift y to match your original range
y = [yi * 30 + 90 for yi in y]

data = {'x': x, 'y': y}


In [175]:
SCALE_FACTOR = 2
PAD = 10

In [176]:
def scaled_rect(xmin, xmax, ymin, ymax, pad=PAD, **kwargs):
    return geom_rect(
        xmin=xmin * SCALE_FACTOR + pad,
        xmax=xmax * SCALE_FACTOR - pad,
        ymin=ymin * SCALE_FACTOR + pad,
        ymax=ymax * SCALE_FACTOR - pad,
        size=pad - 3,
        **kwargs
    )


def labeled_rect(xmin, xmax, ymin, ymax,
                 inner_text='', border_text='',
                 pad=PAD,
                 **kwargs):
    xmin_s = xmin * SCALE_FACTOR + pad
    xmax_s = xmax * SCALE_FACTOR - pad
    ymin_s = ymin * SCALE_FACTOR + pad
    ymax_s = ymax * SCALE_FACTOR - pad

    is_horizontal = (xmax-xmin > (ymax-ymin)/2)

    rect_layer = geom_rect(
        xmin=xmin_s, xmax=xmax_s, ymin=ymin_s, ymax=ymax_s, size = pad-3,
        **kwargs
    )

    # Inner text: centered
    x_center = (xmin_s + xmax_s) / 2
    y_center = (ymin_s + ymax_s) / 2
    inner_layer = geom_text(
        x=x_center, y=y_center, label=inner_text,
        angle=0 if is_horizontal else 90,
        color='darkgray'
    )

    # Border text: near top-right (horizontal) or top-left (vertical)
    if is_horizontal:
        x_border = xmax_s
        y_border = ymin_s + pad / 2 - 2
        angle = 0
        vjust = 0
    else:  # vertical
        x_border = xmin_s - pad / 2 - 3
        y_border = ymin_s
        angle = 90
        vjust = 1

    border_layer = geom_text(
        x=x_border, y=y_border, label=border_text,
        angle=angle, hjust=1, vjust=vjust,
        family='Courier',
        color='black'
    )

    return rect_layer + inner_layer + border_layer

In [177]:
def annotation_line(x, y, xend=None, yend=None, type_arrow='both', linetype='solid'):
    xend = xend if xend is not None else x
    yend = yend if yend is not None else y
    with_arrow = None
    if type_arrow is not None:
        with_arrow = arrow(length=8, ends=type_arrow, type='open', angle=20)

    return geom_segment(x=x, y=y, xend=xend, yend=yend, linetype=linetype, arrow=with_arrow)

In [194]:
def x_ticks(data, y_base=608, tick_length=8):
    ticks = {
        'x': data['x'],
        'xend': data['x'],
        'y': [y_base] * len(data['x']),
        'yend': [y_base + tick_length] * len(data['x']),
    }
    return geom_segment(aes(x='x', y='y', xend='xend', yend='yend'), data=ticks, color='gray50', size=1)

def y_ticks(ymin=208, ymax=574, n_ticks=6, x_base=168, tick_length=8):
    y_ticks = np.linspace(ymin, ymax, n_ticks)

    ticks = {
        'y': y_ticks,
        'yend': y_ticks,
        'x': [x_base] * n_ticks,
        'xend': [x_base + tick_length] * n_ticks,
    }

    return geom_segment(aes(x='x', y='y', xend='xend', yend='yend'), data=ticks, color='gray50', size=1)

In [196]:
COORDS = {
    'PLOT_XMIN': 0, 'PLOT_XMAX': 620, 'PLOT_YMIN': 0, 'PLOT_YMAX': 420,
    'PANEL_XMIN': 88, 'PANEL_XMAX': 508, 'PANEL_YMIN': 90, 'PANEL_YMAX': 304,
    'TITLE_YMIN': 11, 'TITLE_YMAX': 44,
    'SUBTITLE_YMIN': 45, 'SUBTITLE_YMAX': 78,
    'X_TITLE_YMIN': 347, 'X_TITLE_YMAX': 378,
    'Y_TITLE_XMIN': 11, 'Y_TITLE_XMAX': 44, 
    'X_TEXT_XMIN': 98, 'X_TEXT_XMAX': 498, 'X_TEXT_YMIN': 320, 'X_TEXT_YMAX': 334,
    'Y_TEXT_XMIN': 52, 'Y_TEXT_XMAX': 84, 'Y_TEXT_YMIN': 98, 'Y_TEXT_YMAX': 294,
    'INSET_XMIN': 45, 'INSET_XMAX': 517, 'INSET_YMIN': 79, 'INSET_YMAX': 347,
    'CAPTION_YMIN': 378, 'CAPTION_YMAX': 410,
    'LEGEND_XMIN': 532, 'LEGEND_XMAX': 610, 'LEGEND_YMIN': 147, 'LEGEND_YMAX': 244,
    'LEGEND_SPACING_XMIN': 517, 'LEGEND_SPACING_XMAX': 532,
    'FRAME_XMIN': 86.5, 'FRAME_XMAX': 509.5, 'FRAME_YMIN': 86, 'FRAME_YMAX': 305.5,
}

plotting_area = scaled_rect(xmin=COORDS['PLOT_XMIN'], xmax=COORDS['PLOT_XMAX'],
                            ymin=COORDS['PLOT_YMIN'], ymax=COORDS['PLOT_YMAX'],
                            pad=1, fill='white', color='gold', alpha=0.0)

plot_margin = labeled_rect(xmin=COORDS['PLOT_XMIN'], xmax=COORDS['PLOT_XMAX'],
                           ymin=COORDS['PLOT_YMIN'], ymax=COORDS['PLOT_YMAX'],
                           inner_text='', border_text='plot_margin',
                           fill='white', color='lemon_chiffon', alpha=0.9)

title_area = labeled_rect(xmin=COORDS['PANEL_XMIN'], xmax=COORDS['PANEL_XMAX'],
                          ymin=COORDS['TITLE_YMIN'], ymax=COORDS['TITLE_YMAX'],
                          inner_text='TITLE', border_text='plot_title margin',
                          fill='light_blue', color='light_blue', alpha=0.3)

subtitle_area = labeled_rect(xmin=COORDS['PANEL_XMIN'], xmax=COORDS['PANEL_XMAX'],
                             ymin=COORDS['SUBTITLE_YMIN'], ymax=COORDS['SUBTITLE_YMAX'],
                             inner_text='SUBTITLE', border_text='plot_subtitle margin',
                             fill='light_green', color='light_green', alpha=0.3)

y_title_area = labeled_rect(xmin=COORDS['Y_TITLE_XMIN'], xmax=COORDS['Y_TITLE_XMAX'],
                            ymin=COORDS['PANEL_YMIN'], ymax=COORDS['PANEL_YMAX'],
                            inner_text='Y-AXIS TITLE', border_text='axis_title_y margin',
                            fill='peach_puff', color='peach_puff', alpha=0.2)

x_title_area = labeled_rect(xmin=COORDS['PANEL_XMIN'], xmax=COORDS['PANEL_XMAX'],
                            ymin=COORDS['X_TITLE_YMIN'], ymax=COORDS['X_TITLE_YMAX'],
                            inner_text='X-AXIS TITLE', border_text='axis_title_x margin',
                            fill='peach_puff', color='peach_puff', alpha=0.2)

panel_area = labeled_rect(xmin=COORDS['PANEL_XMIN'], xmax=COORDS['PANEL_XMAX'],
                          ymin=COORDS['PANEL_YMIN'], ymax=COORDS['PANEL_YMAX'],
                          inner_text='PLOT\nPANEL', border_text='panel_inset',
                          fill='gray', color='light_gray', alpha=0.5)

y_text_area = labeled_rect(xmin=COORDS['Y_TEXT_XMIN'], xmax=COORDS['Y_TEXT_XMAX'],
                           ymin=COORDS['Y_TEXT_YMIN'], ymax=COORDS['Y_TEXT_YMAX'],
                           inner_text='y-axis labels', border_text='axis_text_y margin',
                           fill='pink', color='light_pink', alpha=0.3)

x_text_area = labeled_rect(xmin=COORDS['X_TEXT_XMIN'], xmax=COORDS['X_TEXT_XMAX'],
                           ymin=COORDS['X_TEXT_YMIN'], ymax=COORDS['X_TEXT_YMAX'],
                           inner_text='x-axis labels', border_text='',
                           pad=0, fill='pink', color='light_pink', alpha=0.3)

x_text_area2 = labeled_rect(xmin=COORDS['X_TEXT_XMIN'], xmax=COORDS['X_TEXT_XMAX'],
                           ymin=COORDS['X_TEXT_YMIN']-PAD, ymax=COORDS['X_TEXT_YMAX']-PAD-4,
                           inner_text='', border_text='axis_text_spacing_x',
                           pad=10, fill='pink', color='light_pink', alpha=0.3)

caption_area = labeled_rect(xmin=COORDS['PANEL_XMIN'], xmax=COORDS['PANEL_XMAX'],
                            ymin=COORDS['CAPTION_YMIN'], ymax=COORDS['CAPTION_YMAX'],
                            inner_text='CAPTION', border_text='plot_caption margin',
                            fill='sky_blue', color='sky_blue', alpha=0.3)

legend_area = labeled_rect(xmin=COORDS['LEGEND_XMIN'], xmax=COORDS['LEGEND_XMAX'],
                           ymin=COORDS['LEGEND_YMIN'], ymax=COORDS['LEGEND_YMAX'],
                           pad=1, inner_text='LEGEND', border_text='',
                           fill='chocolate', color='chocolate', alpha=0.1)

legend_spacing = labeled_rect(xmin=COORDS['LEGEND_SPACING_XMIN'], xmax=COORDS['LEGEND_SPACING_XMAX'],
                              ymin=COORDS['LEGEND_YMIN'], ymax=COORDS['LEGEND_YMAX'],
                              pad=1, inner_text='legend_box_spacing', border_text='',
                              fill='gray93', color='white')

plot_inset_area = labeled_rect(xmin=COORDS['INSET_XMIN'], xmax=COORDS['INSET_XMAX'],
                               ymin=COORDS['INSET_YMIN'], ymax=COORDS['INSET_YMAX'],
                               inner_text='', border_text='plot_inset',
                               fill='white', color='plum', alpha=0.5)

panel_frame = labeled_rect(xmin=COORDS['FRAME_XMIN'], xmax=COORDS['FRAME_XMAX'],
                           ymin=COORDS['FRAME_YMIN'], ymax=COORDS['FRAME_YMAX'],
                           pad=3, inner_text='', border_text='',
                           fill='white', color='gray50')


KeyError: 'Y_TITLE_XMIN'

In [192]:
ggplot(data, aes('x', 'y')) + \
    plot_margin + \
    plotting_area + \
    plot_inset_area + \
    title_area + \
    subtitle_area + \
    y_title_area + \
    y_text_area + \
    x_title_area + \
    x_text_area + \
    x_text_area2 + \
    panel_area + \
    caption_area + \
    legend_spacing + \
    legend_area + \
    annotation_line(x=1058, y=644, xend=1000, yend=613, type_arrow='last') + \
    geom_text(x=1058+3, y=644+3, label='axis tick area', hjust=0, family='Courier', color='black' ) + \
    annotation_line(x=150, y=148, xend=172, yend=194, type_arrow='last') + \
    geom_text(x=150-3, y=148-8, label='axis tick area', hjust=1, family='Courier', color='black' ) + \
    theme_void() + \
    scale_y_reverse() +\
    geom_point(color='dodgerblue', alpha=1) + geom_line(color='dodgerblue', alpha=1) + \
    x_ticks(data) + \
    y_ticks() + \
    ggsize(COORDS['PLOT_XMAX']*SCALE_FACTOR, COORDS['PLOT_YMAX']*SCALE_FACTOR)